In [ ]:
from cracknuts.trace import ScarrTraceDataset
# 加载 scarr 格式的数据集
ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250507110424.zarr')
# ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313022104.zarr')

In [ ]:
# Print dataset info
ds.info()

In [ ]:
from zarr.hierarchy import Group

x: Group = ds.get_origin_data()["/0/0/"]

In [ ]:
# Show trace panel
import cracknuts as cn
pt = cn.panel_trace()
pt.set_trace_dataset(ds)
pt

In [ ]:
# Change trace index.
# show_trace 接收两个索引参数，第一个参数是通道索引，第二个是曲线索引，两个参数支持高级索引
# 需要注意的是通道A和B的名称在数据里为0和1，而其索引则根曲线文件中存储的通道数量有关系，如果只有一个通道时，无论哪个通道索引都为0，如果同时有A B两个通道则A通道索引为0，B通道索引为1，可以通过 ds.channel_names 进行确认当前曲线存储的时哪个通道

# 展示第一通道的前10条曲线
# tp.show_trace[1, :10]
# 展示第一通道的后10条曲线
# tp.show_trace[1, -10:]
# 展示第一通道的 10-20 曲线
# tp.show_trace[1, 10:20]
# 展示所有(两个)通道的前10条中的偶数位曲线
pt.show_trace[:, :10:2]

In [ ]:
# 获取第一通道的前10条曲线数据（np.ndarray）
ds.trace[0, :10]

In [ ]:
# 获取第一通道的前10条明文（当前版本该数据包含明文和密文）
ds.data[0, :10]

In [ ]:
# 加载 numpy 格式 dataset
from cracknuts.trace import NumpyTraceDataset

ds2 = NumpyTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313152352.npy')

In [ ]:
# 获取第二通道的前10条曲线数据（np.ndarray）
ds2.trace[1, :10]

In [ ]:
# 获取第二通道的前10条明文（当前版本该数据包含明文和密文）数据（np.ndarray）
ds2.data[1, :2]

*分析数据*

In [ ]:
from scarr.engines.cpa import CPA as cpa
from scarr.file_handling.trace_handler import TraceHandler as th
from scarr.model_values.sbox_weight import SboxWeight
from scarr.container.container import Container, ContainerOptions
import numpy as np
from smartcard.util import toHexString

In [ ]:
handler2 = th(fileName=r'D:\project\cracknuts\tests\dataset\20250415154328.zarr') 

In [ ]:
model = SboxWeight() 
engine2 = cpa(model)

In [ ]:
container2 = Container(options=ContainerOptions(engine=engine2, handler=handler2), model_positions = [x for x in range(16)])
container2.run()

In [ ]:
results2 = container2.engine.get_result()

In [ ]:
results = np.squeeze(engine2.get_candidate())

In [ ]:
toHexString(list(results))

In [ ]:
result = np.squeeze(results2)
result = result[0]

In [ ]:
candidate = [None for _ in range(result.shape[0])]

for i in range(result.shape[0]):
    candidate[i] = np.unravel_index(np.abs(result[i, :]).argmax(), result[i, :].shape[0:])[0]

In [ ]:
candidates = np.empty(256, dtype=np.float64)
for k in range(256):
    candidates[k] = result[k,candidate[k]]
ck = np.abs(candidates).argmax()

In [ ]:
for _i in range(0, 10):
    print('第',str(_i+1),'个候选值：',hex(np.abs(candidates).argsort()[::-1][_i]),'， 对应的相关系数为：',str(candidates[np.abs(candidates).argsort()[::-1][_i]]))

In [ ]:
backgroud_numpy = np.delete(results2[0, 0, 0, :, :3000], 123, axis=0)

In [ ]:
heighlight_numpy = results2[0, 0, 0, 0x11, :3000][np.newaxis, :]

In [ ]:
results2[0, 0, 0, :, :3000].shape

In [ ]:
np.save('./rs.npy', results2[0, 0, 0, :, :3000])

In [ ]:
result_numpy = np.vstack([backgroud_numpy, heighlight_numpy])

In [ ]:
result_numpy.shape

In [ ]:
pt2 = cn.panel_trace()
pt2.set_numpy_data(result_numpy)
pt2

In [ ]:
pt2.show_all_trace()

In [ ]:
pt2.highlight([0x11])

In [ ]:
pt2.change_percent_range(0, 100)

In [ ]:
pt2.change_range(1000, 1500)